In [1]:
import os
import timm
import torch
import open_clip
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm 
import torchvision.transforms as T
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df.fillna("Unknown")
df.head(2)

,ImageFilePath,Genus,ScientificName
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius,Chlaenius aestivus
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius,Chlaenius aestivus


In [3]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("hf-hub:imageomics/bioclip")

tokenizer = open_clip.get_tokenizer("hf-hub:imageomics/bioclip")

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)


for param in model.parameters() :
    param.requires_grad = False


species_list = df["ScientificName"].unique().tolist() 

text_inputs = tokenizer(species_list).to(device)

correct_predictions = total_images = 0


In [4]:

df["PredSpecies"] = None


for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing Images") : 
    
    image_path = row["ImageFilePath"]

    if not os.path.exists(image_path):
        print(f"File not found: {image_path}")
        continue

    image = Image.open(image_path).convert("RGB")
    image_input = preprocess_val(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)

    
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    
    similarity = (image_features @ text_features.T).squeeze(0)
    best_match_idx = similarity.argmax().item()
    predicted_species = species_list[best_match_idx]

    
    df.at[idx, "PredSpecies"] = predicted_species



Processing Images:   0%|          | 0/11399 [00:00<?, ?it/s]

In [5]:
correct_predictions = (df["PredSpecies"] == df["ScientificName"]).sum()

total_images = len(df)

accuracy = correct_predictions / total_images if total_images > 0 else 0

print(f"BioCLIP Zero-shot SPECIES Classification Accuracy: {accuracy*100:.4f}% ({correct_predictions}/{total_images})")


BioCLIP Zero-shot SPECIES Classification Accuracy: 11.6326% (1326/11399)


In [6]:
df.head(2)

,ImageFilePath,Genus,ScientificName,PredSpecies
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius,Chlaenius aestivus,Elaphropus xanthopus
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius,Chlaenius aestivus,Harpalus compar


In [7]:
df.to_csv('/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/1.BioCLIP-zero-shot-species.csv',index=False)